In [1]:
import numpy as np
import networkx as nx
import datetime
import tqdm
from typing import NamedTuple
import h5py

import time
import random
import pandas as pd
import pickle
from collections import Counter, defaultdict
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bernoulli, binom
sns.set('talk')
sns.set_style('ticks')

#load in search hashtags
with open('../references/search_hashtags.txt', 'r') as f:
    search_hashtags = f.readlines()
    search_hashtags = [i.replace('\n', '') for i in search_hashtags]
    search_hashtags = [i.replace('#', '') for i in search_hashtags]
    search_hashtags = [i.lower() for i in search_hashtags]
    search_hashtags.remove('وأناكمان')

# 0. Import real data and setup

In [2]:
################################################################################
# Set Parameters
################################################################################
group_num = 2
hashtag_split = True
ngram_range = '34'
min_user = 100

overwrite = False
read_in = True

################################################################################
# Set relevant file paths
################################################################################

plot_save_path = f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/results/0{group_num}_group/'

follows_dir = f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/01_raw/0{group_num}_group/'

activity_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/03_processed/activity_counts.hdf5'

peak_analysis_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_peak_analysis.hdf5'

# abm df save path.
abm_processed_df_savepath = f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/ABM_processed_df_group_{group_num}.obj'

graph_savepath = f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/ABM_graph_group_{group_num}.obj'

################################################################################
# Define useful functions
################################################################################

def unit_conv(val):
    return datetime.datetime.strptime('2017-10-16', '%Y-%m-%d') + datetime.timedelta(days=int(val))

def reverse_unit_conv(date):
    return (datetime.datetime.strptime(date, '%Y-%m-%d') - datetime.datetime.strptime('2017-10-16', '%Y-%m-%d')).days

class daterange(NamedTuple):
    start: str
    end: str

def date_to_array_index(date, daterange):
    return (datetime.datetime.strptime(date, '%Y-%m-%d') - datetime.datetime.strptime(daterange.start, '%Y-%m-%d')).days

def group_peaks_and_daterange(peak_analysis_file, group_num):

    #obtain peak times again
    with h5py.File(peak_analysis_file, 'r') as f:
        FAS_peaks = f['peak_detections']
        x = f['segments']['selected_ranges'][int(group_num)-1]
        group_date_range = daterange(
            start = x[0].decode(),
            end = x[1].decode()
        )

        # group_start_index = reverse_unit_conv(group_date_range.start)
        # group_end_index = reverse_unit_conv(group_date_range.end)

        most_prominent_peaks = {}
        for name, h5obj in FAS_peaks.items():

            peak_locations = h5obj['peak_locations']
            peak_locations = [(i,e) for i,e in enumerate(h5obj['peak_locations']) if (unit_conv(e) > datetime.datetime.strptime(group_date_range.start, '%Y-%m-%d')) and (unit_conv(e) < datetime.datetime.strptime(group_date_range.end, '%Y-%m-%d'))]
            peak_indices = [i[0] for i in peak_locations]
            prominences = [element for index, element in enumerate(h5obj['prominences']) if index in peak_indices]
            if len(prominences) == 0:
                continue
            max_prominence = np.argmax(prominences)
            most_prominent_peaks[name] = unit_conv(peak_locations[max_prominence][1])

    daterange_length = (datetime.datetime.strptime(group_date_range.end, '%Y-%m-%d') - datetime.datetime.strptime(group_date_range.start, '%Y-%m-%d')).days

    return most_prominent_peaks, group_date_range, daterange_length

################################################################################
# Read in peaks
################################################################################

most_prominent_peaks, group_date_range, daterange_length = group_peaks_and_daterange(peak_analysis_file, group_num)


################################################################################
# Determine if ABM df has already been processed.
################################################################################

if (os.path.isfile(abm_processed_df_savepath) and overwrite) or not os.path.isfile(abm_processed_df_savepath):

    print('overwriting or writing for the first time')

    # read df raw for ABM
    stats_df_save_dir = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/'
    df_filename = os.path.join(stats_df_save_dir, f'ABM_raw_df_group_{group_num}.obj')
    stats_filename = os.path.join(stats_df_save_dir, f'ABM_stats_df_group_{group_num}.obj')

    if os.path.isfile(df_filename):
        print('reading in df')
        with open(df_filename, 'rb') as f:
            df = pickle.load(f)
    if os.path.isfile(stats_filename):
        print('reading in stats_df')
        with open(stats_filename, 'rb') as f:
            stats_df = pickle.load(f)

    print('N.B. users are not included in stats df because in creating the activity counts users were split into before and after peak interactions')

    print(f'Length of df: {len(df)}')
    unique_author_stats_df_count = len(stats_df['author_id'].unique())
    print(f'Number of unique authors in stats df: {unique_author_stats_df_count}')
    unique_author_df_count = len(df['author_id'].unique())
    print(f'Number of unique authors in df: {unique_author_df_count}')

    # generate ht column
    df_colnames = df.columns
    vocab_colnames = [i for i in df_colnames if i.startswith('vocab')][::-1]
    def process_row_ht(row):
        for col in vocab_colnames:
            if col == 'vocab:#timesup':
                continue
            if row[col] == 1:
                return col.split('#')[-1]
        return None

    df['ht'] = df.apply(process_row_ht, axis=1)
    df['ht'] = df['ht'].fillna('metoo')

    df = df.merge(stats_df, on=['author_id','ht'], how='right')

    # incorporate primary ht
    with open(f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/03_processed/primary_ht_global.obj', 'rb') as f:
        user_order, res = pickle.load(f)

    unknown_count = 0
    def process_primary_res(author_id):
        global unknown_count
        if author_id not in user_order:
            # print(f'{author_id} not in users')
            unknown_count += 1
            return 'metoo'
        return search_hashtags[np.argmax(res[user_order.index(author_id),:])]


    df['primary_ht'] = df['author_id'].map(df.groupby('author_id').apply(lambda x: process_primary_res(x.name)))
    print(f'Number of unknown primary hashtags for users: {unknown_count}')

    df['interacted_users'].loc[df['interacted_users'].isnull()] = df['interacted_users'].loc[df['interacted_users'].isnull()].apply(lambda x: [])

    with open(abm_processed_df_savepath, 'wb') as f:
        pickle.dump(df, f)


elif os.path.isfile(abm_processed_df_savepath) and read_in:
    print('reading in')
    with open(abm_processed_df_savepath, 'rb') as f:
        df = pickle.load(f)

################################################################################
# Generate the 
################################################################################

temp_df = df.groupby('author_id').first()

reading in


In [3]:
# archive:

# df[['text','lang']]

#### Language detection for individual tweets - possibly something to pass to ARC.
# from langdetect import detect
# import langdetect

# def proper_detect(text):
#     try:
#         return detect(text)
#     except:
#         return None

# df['text'].apply(proper_detect)

# 1. Make the user network

Grow a graph using the Barabási-Albert preferential attachment model.

A graph of `n` nodes is grown by attaching new nodes each with `m` edges that are preferentially attached to existing nodes with high degree.

https://networkx.org/documentation/networkx-1.9.1/reference/generated/networkx.generators.random_graphs.barabasi_albert_graph.html

In [4]:
def generate_network(df, follows_dir = follows_dir):

    # construct graph
    G = nx.DiGraph()

    # add nodes to graph
    unique_users = df['author_id'].unique()

    # for the most common hasthags used by an interacted user
    filtered_ht = df.groupby('author_id')['ht'].agg(pd.Series.mode).to_dict()

    # attributes
    # G = nx.path_graph(3)
    # attrs = {0: {"attr1": 20, "attr2": "nothing"}, 1: {"attr2": 3}}
    # nx.set_node_attributes(G, attrs)

    # switch correct type of graph being constructed:

    # OLD AS OF 11 MAY 2022
    # if gen_from == 'interaction':

    #     # first add all nodes
    #     for row in df.itertuples(index=False):
    #         G.add_node(row.author_id)
    #         if type(row.interacted_users) == float and np.isnan(row.interacted_users):
    #             continue
    #         for interacted_user in row.interacted_users:
    #             if interacted_user not in unique_users:
    #                 continue
    #             G.add_edge(row.author_id, interacted_user)

    #     print(2*G.number_of_edges() / float(G.number_of_nodes()))

    # sanity check: followers data is collected
    assert os.path.isdir(follows_dir)

    successful_follows=0
    # extract users whom a user follows and following
    for user_id in tqdm.tqdm(unique_users):
        # print(f'Processing {user_id}')
        follows_filepath = os.path.join(follows_dir, f'following_{user_id}.txt')
        try:
            total_edges_to_add = df[df['author_id'] == user_id]['interacted_users'].sum()
        except:
            print(user_id)
        if isinstance(total_edges_to_add, int):
            total_edges_to_add = np.array([])
        total_edges_to_add = np.intersect1d(total_edges_to_add, unique_users)

        if os.path.isfile(follows_filepath):
            try:
                edges_to = pd.read_table(follows_filepath).values.flatten().astype(str)
                successful_follows+=1
                new_total_edges_to_add = np.union1d(total_edges_to_add, edges_to)
                new_total_edges_to_add = np.intersect1d(new_total_edges_to_add, unique_users)
                length_diff = len(new_total_edges_to_add) - len(total_edges_to_add)
                assert length_diff >= 0
                # print(f'Length diff: {length_diff} for {user_id}')
                total_edges_to_add = new_total_edges_to_add
            except pd.errors.EmptyDataError:
                pass

        for interacted_user in total_edges_to_add:
            ht = filtered_ht[interacted_user]
            G.add_edge(user_id, interacted_user, ht=ht)

    print(f'Total successfully read follows: {successful_follows}')

    # then add attributes:

    # for k,v in user_edges.items():

    #     # add k
    #     G.add_node(str(k))

    #     v_counter = Counter(v)
    #     v_edge_list = [(str(k), str(node), count) for node,count in v_counter.items() if str(node) in user_edges_keys]

    #     G.add_weighted_edges_from(v_edge_list)

    # with open(graph_object_file, 'wb') as f:
    #     pickle.dump(G, f)
    return G


if os.path.isfile(graph_savepath):
    print('loading in')
    with open(graph_savepath, 'rb') as f:
        G = pickle.load(f)
else:
    G = generate_network(df)
    with open(graph_savepath, 'wb') as f:
        pickle.dump(G,f)

loading in


In [5]:
# df[df['author_id']=='16996244']['interacted_users']

In [6]:
# positions = nx.spring_layout(G)
# nx.draw(G, node_size=25, pos=positions)

In [7]:
# degrees        = dict(G.degree()).values()
# sorted_degrees = sorted(degrees)[::-1]

# fig, ax = plt.subplots(figsize=(20,5))

# ax.bar(x=range(len(sorted_degrees)), height=sorted_degrees)

# plt.xlabel('Nodes')
# plt.ylabel('Node degree')
# plt.xlim(-5,len(degrees)+5)

# plt.show()

## Get statistics from data

# 2. Make the agents

In [8]:
class Agent(object): 

    # initialise internal variables
    def __init__(self, ID, df, search_hashtags):
        self.ID = ID
        self.supporting_metoo = False    # initial assumption.
        self.supporting_metoo_dict = {i:0 for i in search_hashtags}
        self.interacts_with   = []       # I wrote this to represent a symmetric interaction, rather than following.
        # self.interaction_counter = defaultdict(int)    # counts interactions
        self.forget_all_interactions()
        # print(ID)
        # row = df[df['author_id'] == ID].iloc[0,:]
        # self.total_hashtags = row.author_total_hashtags
        self.primary_ht = df.loc[ID,'primary_ht']
        self.support_tracker = np.zeros(shape=(35,1))
        self.activity_tracker = np.zeros(shape=(35,1))
        self.individual_propensity = np.zeros(shape=(35,1))

    def update_self_support_and_propensity(self):

        # support_update_array = np.zeros(shape=(35,1))
        # individual_prop_update_array = np.zeros(shape=(35,1)) 
        # print('processing propensities')

        # index=0
        # for k,v in self.propensity_params.items():
        #     support_update_array[index,0] = self.supporting_metoo_dict[k]
        #     individual_prop_update_array[index,0] = (1-v)**(support_update_array[index,0])
        #     index += 1

        # start_time = time.time()

        support_update_array = np.array(list(self.supporting_metoo_dict.values())).reshape(-1,1)
        # individual_prop_update_array = (1-np.array(list(self.propensity_params.values())))**support_update_array

        # individual_prop_update_array = individual_prop_update_array.reshape(-1,1)

        self.support_tracker = np.hstack((self.support_tracker, support_update_array))
        # self.individual_propensity = np.hstack((self.individual_propensity, individual_prop_update_array))

        # timetaken = time.time() - start_time
        # print(f'end sim. Time taken: {timetaken}')

    def simulate(self):

        # print('updating')
        self.update_self_support_and_propensity()
        # print('updating done')

        # update_array = np.zeros(shape=(35,1))
        # for index, v in enumerate(self.individual_propensity[:,-1]):
        #     b = bernoulli(v)
        #     update_array[index, 0] = 1-b.rvs(1)[0]
        # update_array = update_array.reshape(-1,1)
        # self.activity_tracker = np.hstack((self.activity_tracker,update_array))

    def interact(self, other):

        # Keep track of interactions with others.
        # This step is asymmetric: 'self' keeps track, but 'other' does not.

        if other.ID not in self.interaction_counter:
            self.interaction_counter[other.ID]  = 1
        else:
            self.interaction_counter[other.ID] += 1

    # How do they behave?
    # How do agents change?
    def maybe_join(self, other, filtered_ht_dict, interact_threshold = 1, model_num = None, verbose=False):


        # Model num. This is just for ease of reproducibility. model_num being None means take the latest (probably most complex) ABM.

        if model_num == 1:

            # Simplest model. No 
            if self.supporting_metoo == False and \
                other.supporting_metoo == True and \
                self.interaction_counter[other.ID] > interact_threshold:

                self.supporting_metoo = True

                if verbose:
                    print(f'Agent {self.ID} has spoken a lot to Agent {other.ID} and now supports {other.primary_ht}')

        elif model_num == 2:

            # Model 2:
            # still have singular 'metoo' supporting
            # ADD different primary language requirement.
            if  self.supporting_metoo == False and \
                other.supporting_metoo == True and \
                other.primary_ht != self.primary_ht and \
                self.interaction_counter[other.ID] > interact_threshold:

                self.supporting_metoo = True

                if verbose:
                    print(f'Agent {self.ID} has spoken a lot to Agent {other.ID} and now supports {other.primary_ht}')

        elif model_num == 3:

            # Model 3:
            # still have singular 'metoo' supporting
            # and different language requirement.
            # ADD minimum reciprocal interaction
            if  self.supporting_metoo == False and \
                other.supporting_metoo == True and \
                other.primary_ht != self.primary_ht and \
                self.interaction_counter[other.ID] > interact_threshold and \
                other.interaction_counter[self.ID] > interact_threshold:

                self.supporting_metoo = True

                if verbose:
                    print(f'Agent {self.ID} has spoken a lot to Agent {other.ID} and now supports {other.primary_ht}')

        elif model_num == 4:

            # Model 4:
            # ADD metoodict
            # and different language requirement.
            # and minimum reciprocal interaction.
            #
            # Now there is the possibility of those who are 'supporting metoo' already in one language to be influenced to support another language.
            #
            # This model only allows for one user to influence another on their primary hashtag.

            # if  (self.supporting_metoo is False) and \
            if  (self.interaction_counter[other.ID] > interact_threshold) and \
                (other.primary_ht != self.primary_ht) and \
                (self.supporting_metoo_dict[other.primary_ht] == 0) and \
                (other.interaction_counter[self.ID] > interact_threshold):

                self.supporting_metoo_dict[other.primary_ht] += 1

                if verbose:
                    print(f'Agent {self.ID} has spoken a lot to Agent {other.ID} and now supports {other.primary_ht}')

        elif model_num == 5:

            # Model 5:
            # use metoodict
            # and different language requirement.
            # and minimum reciprocal interaction.
            #
            # ADD ability to influence other users within your community too.
            #
            # This model only allows for one user to influence another on their primary hashtag.

            # if  (self.supporting_metoo is False) and \
            if  (self.interaction_counter[other.ID] > interact_threshold) and \
                (other.primary_ht != self.primary_ht) and \
                (self.supporting_metoo_dict[other.primary_ht] == 0) and \
                (other.interaction_counter[self.ID] > interact_threshold):

                self.supporting_metoo_dict[other.primary_ht] += 1

                if verbose:
                    print(f'Agent {self.ID} has spoken a lot to Agent {other.ID} and now supports {other.primary_ht}')

            elif (self.interaction_counter[other.ID] > interact_threshold) and \
                 (other.primary_ht == self.primary_ht) and \
                 (other.interaction_counter[self.ID] > interact_threshold):

                samplelist = [k for k,v in other.supporting_metoo_dict.items() if v>0]

                if samplelist:
                    # select a random value within
                    sampled_ht_for_influence = random.choices(
                        samplelist,
                        [v for _,v in other.supporting_metoo_dict.items() if v>0],
                        k=1
                    )

                    self.supporting_metoo_dict[sampled_ht_for_influence[0]] += 1

                if verbose:
                    print(f'Agent {self.ID} has influenced someone of their own primary ht community.')


        elif model_num is None:
            pass

    def forget_all_interactions(self):
        self.interaction_counter = defaultdict(int)

    def forget_support_metoo(self):
        self.supporting_metoo = False
        self.supporting_metoo_dict = {i:0 for i in search_hashtags}

In [9]:
# Assemble network of agents
# agents = { ID: Agent(ID, df) for ID in df['author_id'].unique() }
# this is extremely slow
%load_ext line_profiler

# temp_groupby_for_agent_creation
# temp_df = df.groupby('author_id').first()

def produce_agents():
    agents = {}
    for user in tqdm.tqdm(list(df['author_id'].unique())):
        agents[user] = Agent(user,temp_df, search_hashtags)

    return agents

# %lprun -f Agent.__init__ produce_agents()

agents_overwrite = True
agents_read_in = True
agents_savepath = f'/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/ABM_agents_group_{group_num}.obj'

if os.path.isfile(agents_savepath) and agents_overwrite:
    print('File exists and overwriting')
    agents = produce_agents()
    with open(agents_savepath, 'wb') as f:
        pickle.dump(agents, f)
elif os.path.isfile(agents_savepath) and agents_read_in:
    print('reading in')
    with open(agents_savepath, 'rb') as f:
        agents = pickle.load(f)
elif not os.path.isfile(agents_savepath):
    print('producing agents for the first time')
    agents = produce_agents()
    with open(agents_savepath, 'wb') as f:
        pickle.dump(agents, f)

print('creating agents complete')

File exists and overwriting


100%|██████████| 12120/12120 [00:00<00:00, 60087.40it/s]


creating agents complete


In [10]:
for user_id, agent in agents.items():
    agent.forget_all_interactions()
    agent.forget_support_metoo()
for edge in G.edges():
    i,j = edge
    agents[i].interacts_with.append(j)

In [11]:
print( 'For instance, agent 12 interacts with', agents[list(agents.keys())[11]].interacts_with )

For instance, agent 12 interacts with ['2188044039', '266215308', '3023497329', '473782165', '4897361049', '80820758', '815500888634785792', '89784381', '932933255258615809', '947601090']


# 2.5: Intermediate checks with likes std

In [ ]:
# intermediate: check correlation between likes std and activity tout court:

act_val = {}
for user_id, agent in agents.items():
    # obtain user activity

    act_val[user_id] = {}
    with h5py.File(activity_file, 'r') as f:
        activity = f[f'group_{group_num}'][user_id]['hashtagged'][:]
        feature_order = f[f'group_{group_num}'][user_id]['hashtagged'].attrs['feature_order']
        feature_order = feature_order.split(';')
        # act_val[user_id] = np.sum(activity[:,-int(daterange_length/2):])

        for hashtag_in_period in most_prominent_peaks:
            hashtag_in_period_index = feature_order.index(hashtag_in_period)

            # obtain the index offset from the detected peak of the hashtag to collect initial time window.
            peak_index_index = (datetime.datetime.strptime(group_date_range.end, '%Y-%m-%d')-most_prominent_peaks[hashtag_in_period]).days
            # offset_index -= peak_delta_init
            # offset_index = max(0,offset_index)+1
            # print(f'Offset for {hashtag_in_period} is {offset_index}')

            act_val[user_id][hashtag_in_period_index]= np.sum(activity[hashtag_in_period_index,-peak_index_index-1:])

act_val = pd.DataFrame.from_dict(act_val, orient='index').reset_index()
act_val.columns = ['user_id'] + list(most_prominent_peaks.keys())

# 3. Run the model

First model:

In [16]:
#model parameters:

# N.B. these thresholds are > not >=
interact_threshold = 1
initial_activity_threshold = 2
peak_delta_init = 7
model_num = 4
interact_prob = 0.9

search_hashtag_propensity = {
    'metoo': 0.1,
    'balancetonporc': 0.1,
    'moiaussi': 0.1,
    'نه_یعنی_نه': 0.1,
    '米兔': 0.1,
    '我也是': 0.1,
    'gamani': 0.1,
    'tôicũngvậy': 0.1,
    '私も': 0.1,
    'watashimo': 0.1,
    '나도': 0.1,
    '나도당했다': 0.1,
    'גםאנחנו': 0.1,
    'ятоже': 0.1,
    'ricebunny': 0.1,
    'enazeda': 0.1,
    'anakaman': 0.1,
    'yotambien': 0.1,
    'sendeanlat': 0.1,
    'kutoo': 0.1,
    'withyou': 0.1,
    'wetoo': 0.1,
    'cuentalo': 0.1,
    'quellavoltache': 0.1,
    'niunamenos': 0.1,
    'woyeshi': 0.1,
    'myharveyweinstein': 0.1,
    'noustoutes': 0.1,
    'stilleforopptak': 0.1,
    'nårdansenstopper': 0.1,
    'nårmusikkenstilner': 0.1,
    'memyös': 0.1,
    'timesup': 0.1,
    'niere': 0.1,
    'jotambe': 0.1
}

In [13]:
def reset_abm(agents, initial_activity_threshold, search_hashtag_propensity, peak_delta_init = 10):

    # First, reset everyone's memory of their interactions
    for user_id, agent in agents.items():
        agent.forget_all_interactions()
        agent.forget_support_metoo()
        agent.propensity_params = search_hashtag_propensity


    # Alternate second step: activate everyone above a certain activity threshold before the daterange. This activation will be done by primary language they have expressed some 
    pre_val = {}
    for user_id, agent in agents.items():

        pre_val[user_id] = {}

        # obtain user activity
        with h5py.File(activity_file, 'r') as f:
            activity = f[f'group_{group_num}'][user_id]['hashtagged'][:]
            feature_order = f[f'group_{group_num}'][user_id]['hashtagged'].attrs['feature_order']
            feature_order = feature_order.split(';')

            # obtain values for the hashtags that have peaks in this time period
            for hashtag_in_period in most_prominent_peaks:
                hashtag_in_period_index = feature_order.index(hashtag_in_period)

                # obtain the index offset from the detected peak of the hashtag to collect initial time window.
                offset_index = (most_prominent_peaks[hashtag_in_period] - datetime.datetime.strptime(group_date_range.start, '%Y-%m-%d')).days
                offset_index -= peak_delta_init
                offset_index = max(0,offset_index)+1
                # print(f'Offset for {hashtag_in_period} is {offset_index}')

                pre_val[user_id][hashtag_in_period_index]= np.sum(activity[hashtag_in_period_index,:offset_index])

    pre_val = pd.DataFrame.from_dict(pre_val, orient='index').reset_index()
    pre_val.columns = ['user_id'] + list(most_prominent_peaks.keys())

    init_count = set()
    for _, row in pre_val.iterrows():
        for hashtag_in_period in most_prominent_peaks:
            if row[hashtag_in_period] > initial_activity_threshold:
                init_count.add(row['user_id'])
                agent_in_question = agents[row['user_id']]
                agent_in_question.supporting_metoo=True
                agent_in_question.supporting_metoo_dict[hashtag_in_period] = True

    print(f'Total initally set to support: {len(init_count)}')

    return pre_val


In [14]:
pre_val = reset_abm(agents, initial_activity_threshold=initial_activity_threshold,
search_hashtag_propensity=search_hashtag_propensity,
peak_delta_init=peak_delta_init)

Total initally set to support: 337


In [15]:
pre_val.iloc[:,1:].sum(axis=0)

metoo         4862.0
moiaussi        69.0
niunamenos      42.0
noustoutes     177.0
wetoo          287.0
ятоже            0.0
米兔               0.0
dtype: float64

In [24]:
def run_model(
        agents,
        filtered_ht_dict,
        daterange_length,
        model_num,
        interact_prob,
        verbose = False
    ):

    for time in tqdm.tqdm(range(daterange_length)):
        if verbose:
            print(f'Starting interactions on day {time+1}')
        for _, agent in agents.items():

            # pick a random person that the agent interacts with
            try:

                if np.random.uniform()<=interact_prob:
                    other_agent = agents[np.random.choice(agent.interacts_with)]

                    # interact with them
                    agent.interact(other_agent)

                    # if you've interacted with them many times recently, say something
                    agent.maybe_join(
                        other_agent,
                        filtered_ht_dict,
                        interact_threshold,
                        model_num = model_num,
                        verbose=verbose)

                    agent.simulate()

            except ValueError:
                continue

    return agents


filtered_ht_dict = df.groupby('author_id')['ht'].agg(pd.Series.mode).to_dict()

# some have multiple for mode, just take first one as an approximation now:
for k,v in filtered_ht_dict.items():
    if isinstance(v, np.ndarray):
        filtered_ht_dict[k] = v[0]

modelled_agents = run_model(
            agents,
            filtered_ht_dict,
            daterange_length,
            model_num,
            interact_prob,
            verbose = True
        )

  0%|          | 0/42 [00:00<?, ?it/s]

Starting interactions on day 1


  2%|▏         | 1/42 [00:00<00:25,  1.63it/s]

Starting interactions on day 2
Agent 1001652660452380673 has influenced someone of their own primary ht community.
Agent 1006618246294876160 has influenced someone of their own primary ht community.
Agent 1012594398272471040 has influenced someone of their own primary ht community.
Agent 1015937448793157632 has influenced someone of their own primary ht community.
Agent 101674815 has influenced someone of their own primary ht community.
Agent 1017141534880403456 has influenced someone of their own primary ht community.
Agent 1018598912 has influenced someone of their own primary ht community.
Agent 1025113895415889921 has influenced someone of their own primary ht community.
Agent 1037560485707567104 has influenced someone of their own primary ht community.
Agent 1041020734871560192 has influenced someone of their own primary ht community.
Agent 1044068816177717248 has influenced someone of their own primary ht community.
Agent 1045505730772709376 has influenced someone of their own pr

  5%|▍         | 2/42 [00:01<00:23,  1.69it/s]

...


Agent 906874610070810624 has influenced someone of their own primary ht community.
Agent 90725961 has influenced someone of their own primary ht community.
Agent 907293930936553472 has influenced someone of their own primary ht community.
Agent 907555871793786880 has influenced someone of their own primary ht community.
Agent 907648749941354496 has influenced someone of their own primary ht community.
Agent 907952401486725122 has influenced someone of their own primary ht community.
Agent 909313174054342657 has spoken a lot to Agent 2370435626 and now supports noustoutes
Agent 910264616831373315 has influenced someone of their own primary ht community.
Agent 911300651245867008 has influenced someone of their own primary ht community.
Agent 912087914212990976 has influenced someone of their own primary ht community.
Agent 912242053 has influenced someone of their own primary ht community.
Agent 912656819356549123 has influenced someone of their own primary ht community.
Agent 914093653

 98%|█████████▊| 41/42 [00:47<00:01,  1.71s/it]


Agent 95054282 has influenced someone of their own primary ht community.
Agent 951108644455694338 has influenced someone of their own primary ht community.
Agent 951629597875073025 has influenced someone of their own primary ht community.
Agent 952267723 has influenced someone of their own primary ht community.
Agent 95244774 has influenced someone of their own primary ht community.
Agent 953663515037876224 has influenced someone of their own primary ht community.
Agent 957051510503952384 has influenced someone of their own primary ht community.
Agent 957372954 has influenced someone of their own primary ht community.
Agent 959724885504286720 has influenced someone of their own primary ht community.
Agent 9616062 has influenced someone of their own primary ht community.
Agent 962153872616837120 has influenced someone of their own primary ht community.
Agent 962289612981878784 has influenced someone of their own primary ht community.
Agent 962925801812328448 has influenced someone of t

100%|██████████| 42/42 [00:49<00:00,  1.17s/it]


Agent 990218675964686337 has influenced someone of their own primary ht community.
Agent 991245272192872448 has influenced someone of their own primary ht community.
Agent 991949448908423168 has influenced someone of their own primary ht community.
Agent 993302124091117573 has spoken a lot to Agent 459681852 and now supports metoo
Agent 995288225554952193 has influenced someone of their own primary ht community.
Agent 995328867467382784 has influenced someone of their own primary ht community.
Agent 995719854668828672 has influenced someone of their own primary ht community.
Agent 996293997181071360 has influenced someone of their own primary ht community.
Agent 998366525793824769 has influenced someone of their own primary ht community.
Agent 998524191652569089 has influenced someone of their own primary ht community.
Agent 999313801538306048 has influenced someone of their own primary ht community.
Agent 999642446723248129 has influenced someone of their own primary ht community.
Ag

# 4. Display Results

In [25]:
def print_abm_results(agents_dict, model_num=model_num):

    output_dict={}

    total = len(agents_dict)
    # num_supporting = 0
    # num_not_supporting = 0

    for user_id, agent in agents_dict.items():
        output_dict[user_id] = agent.supporting_metoo_dict

    output_df = pd.DataFrame.from_dict(output_dict, orient='index').reset_index()

    num_supporting = output_df.iloc[:,1:].sum(axis=0)
    num_supporting = num_supporting[num_supporting>0]
    num_not_supporting = (output_df==False).sum(axis=0)

    print(num_supporting)
    # print(num_not_supporting)
    return output_df

In [37]:
np.array(list(x.supporting_metoo_dict.values())).reshape(-1,1)

NameError: name 'x' is not defined

In [ ]:
x = agents['4905648128']
np.hstack((x.support_tracker,
        np.array(list(x.supporting_metoo_dict.values()))))

In [26]:
res = print_abm_results(modelled_agents)

metoo             4699
balancetonporc     496
moiaussi            59
yotambien          111
sendeanlat          30
kutoo              491
withyou            167
wetoo              153
cuentalo           506
quellavoltache      82
niunamenos        1044
woyeshi             97
noustoutes         599
timesup            397
dtype: object


In [27]:
# compare with actual activity

# define active at the end as having above 10? or somewhere along the distribution?

act_val = {}
for user_id, agent in agents.items():
    # obtain user activity

    act_val[user_id] = {}
    with h5py.File(activity_file, 'r') as f:
        activity = f[f'group_{group_num}'][user_id]['hashtagged'][:]
        feature_order = f[f'group_{group_num}'][user_id]['hashtagged'].attrs['feature_order']
        feature_order = feature_order.split(';')
        # act_val[user_id] = np.sum(activity[:,-int(daterange_length/2):])

        for hashtag_in_period in most_prominent_peaks:
            hashtag_in_period_index = feature_order.index(hashtag_in_period)

            # obtain the index offset from the detected peak of the hashtag to collect initial time window.
            peak_index_index = (datetime.datetime.strptime(group_date_range.end, '%Y-%m-%d')-most_prominent_peaks[hashtag_in_period]).days
            # offset_index -= peak_delta_init
            # offset_index = max(0,offset_index)+1
            # print(f'Offset for {hashtag_in_period} is {offset_index}')

            act_val[user_id][hashtag_in_period_index]= np.sum(activity[hashtag_in_period_index,-peak_index_index-1:])

act_val = pd.DataFrame.from_dict(act_val, orient='index').reset_index()
act_val.columns = ['user_id'] + list(most_prominent_peaks.keys())

# act_val = pd.DataFrame.from_dict(act_val, orient='index').reset_index()
# act_val.columns = ['user_id', 'val']

# (act_val['val']>10).sum()


In [30]:
(act_val.iloc[:,1:]>0).sum(axis=0)

metoo         3912
moiaussi       138
niunamenos    1677
noustoutes    1626
wetoo           84
ятоже            3
米兔              22
dtype: int64

# 5. Visualise

In [70]:
res[res['kutoo']>0]['kutoo'].sort_values()

64      True
6368    True
6278       1
6201    True
6037    True
        ... 
4661       5
1158       5
3487       6
5093       7
4563      17
Name: kutoo, Length: 97, dtype: object

In [71]:
res[res['metoo']>0]['metoo'].sort_values(ascending=False)

1078      19
1685       7
7060       5
6440       4
4764       4
        ... 
3462    True
3643    True
3663    True
3688       1
5035    True
Name: metoo, Length: 251, dtype: object

In [72]:
res[res['kutoo']>0]['kutoo'].sort_values(ascending=False)

4563      17
5093       7
3487       6
4661       5
1158       5
        ... 
2442    True
2284       1
2105    True
1931    True
4543    True
Name: kutoo, Length: 97, dtype: object

In [58]:
res[res['niunamenos']>0]['niunamenos'].sort_values(ascending=False)

7489       2
17      True
6949    True
6626    True
6429    True
        ... 
3235    True
3234    True
3037    True
2981    True
9463    True
Name: niunamenos, Length: 86, dtype: object

In [39]:
agents['92263198'].support_tracker[19,:]

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [38]:
search_hashtags.index('kutoo')

19

In [90]:
res

,index,metoo,balancetonporc,moiaussi,نه_یعنی_نه,米兔,我也是,gamani,tôicũngvậy,私も,...,woyeshi,myharveyweinstein,noustoutes,stilleforopptak,nårdansenstopper,nårmusikkenstilner,memyös,timesup,niere,jotambe
0,1000012910,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000030383969316865,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000044199436259328,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000093693385347072,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000433909472092160,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12115,999816789675593728,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12116,999922232523603969,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12117,999927005503045633,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12118,999934418784485376,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
for ht, _ in most_prominent_peaks.items():
    abm_res = res[res[ht]>0][['index', ht]]
    act_res = act_val[act_val[ht]>3][['user_id',ht]]

    overlap = set(abm_res['index']) & set(act_res['user_id'])

    print(f'For {ht}: overlap = {len(overlap)}, total = {len(act_res)}')

For metoo: overlap = 108, total = 466
For moiaussi: overlap = 2, total = 8
For niunamenos: overlap = 2, total = 164
For noustoutes: overlap = 27, total = 363
For wetoo: overlap = 2, total = 7
For ятоже: overlap = 0, total = 0
For 米兔: overlap = 0, total = 0
